In [ ]:
import astropy
import astropy.time as atime
import astropy.timeseries as timeseries
import astropy.units as u
import astropy.visualization
import numpy as np
import pandas as pd

astropy.visualization.quantity_support()

import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['TeX Gyre Heros', 'Helvetica', 'Arial', 'sans serif']
plt.rcParams['font.cursive'] = ['TeX Gyre Chorus', 'Apple Chancery', 'cursive']
plt.rcParams['mathtext.fontset'] = 'stixsans'

for p in matplotlib.font_manager.findSystemFonts(fontpaths=[
    '/usr/share/texmf/fonts/opentype/', '/usr/share/fonts/opentype']):
    matplotlib.font_manager.fontManager.addfont(p)

In [ ]:
ASASsn_table = pd.read_csv('../data/light_curve_8ee0099b-8272-46d9-be75-2c5a50561150.csv')
ASASsn_table

In [ ]:
ASASsn_g = ASASsn_table.query('Filter == "g" & mag < 90')
ASASsn_g_time = atime.Time(ASASsn_g['HJD'], format='jd', scale='utc')

ls_g = timeseries.LombScargle(
    t=ASASsn_g_time,
    y=np.array(ASASsn_g['mag']),
    dy=np.array(ASASsn_g['mag_err']),
    # normalization='psd',
)

max_period = (np.max(ASASsn_g['HJD']) - np.min(ASASsn_g['HJD'])) * u.day
freq, power = ls_g.autopower(minimum_frequency=1/max_period, maximum_frequency=1/(1*u.day))

fig, axs = plt.subplots(2, sharex=True, figsize=(4, 6), dpi=144)
ax = axs[0]
ax.plot(1/freq, power)
ax.set_xscale('log')
ax.set_ylim(0., 0.034)
ax.set_title('ASASSN Lomb-Scargle')
ax.hlines(ls_g.false_alarm_level([0.01, 0.05]), 2, 1000, linestyle='--', colors='gray')
ax.vlines([17.5, 90], 0, 0.4, linestyle='--', colors='gray')


window_fn = timeseries.LombScargle(
    t=ASASsn_g_time,
    y=np.ones_like(ASASsn_g_time),
    fit_mean=False,
    center_data=False,
    # normalization='psd',
)
wn_power = window_fn.power(freq)
ax = axs[1]
ax.plot(1/freq, wn_power)
ax.set_title('ASASSN window function')
ax.vlines([17.5, 90], -0.1, 0.42, linestyle='--', colors='gray')
ax.set_ylim(0, 0.4)

ax.set_xlim(2, 1000)


fig.tight_layout()

ls_g.false_alarm_level([0.01, 0.05])

In [ ]:
ASASsn_V = ASASsn_table.query('Filter == "V" & mag < 90')
ASASsn_V_time = atime.Time(ASASsn_V['HJD'], format='jd', scale='utc')

ls_V = timeseries.LombScargle(
    t=ASASsn_V_time,
    y=np.array(ASASsn_V['mag']),
    dy=np.array(ASASsn_V['mag_err']),
    # normalization='psd',
)

max_period_V = (np.max(ASASsn_V['HJD']) - np.min(ASASsn_V['HJD'])) * u.day
freq_V, power_V = ls_V.autopower(minimum_frequency=1/max_period_V, maximum_frequency=1/(1*u.day))

fig, axs = plt.subplots(2, sharex=True, figsize=(4, 6), dpi=144)
ax = axs[0]
ax.plot(1/freq_V, power_V)
ax.set_xscale('log')
ax.set_ylim(0., 0.1)
ax.set_title('ASASSN Lomb-Scargle $V$')
ax.hlines(ls_V.false_alarm_level([0.01, 0.05]), 2, 1000, linestyle='--', colors='gray')
ax.vlines([17.5, 90], 0, 0.4, linestyle='--', colors='gray')


window_fn_V = timeseries.LombScargle(
    t=ASASsn_V_time,
    y=np.ones_like(ASASsn_V_time),
    fit_mean=False,
    center_data=False,
    # normalization='psd',
)
wn_power_V = window_fn_V.power(freq_V)
ax = axs[1]
ax.plot(1/freq_V, wn_power_V)
ax.set_title('ASASSN window function')
ax.vlines([17.5, 90], -0.1, 0.42, linestyle='--', colors='gray')
ax.set_ylim(0, 0.1)

ax.set_xlim(2, 1000)


fig.tight_layout()

ls_g.false_alarm_level([0.01, 0.05])

In [ ]:
def get_peaks(freq, power, num_peaks=5):
    sort_index = np.argsort(power)
    copy_power = np.zeros(len(power)+2, dtype=float)
    copy_power[1:-1] = power
    # Horrible way of testing for local maxima.
    is_peak = np.logical_and(power > copy_power[:-2], power > copy_power[2:])
    peak_order = np.argsort(power[is_peak])
    peaks = freq[is_peak][peak_order[-1:-num_peaks-1:-1]]
    return peaks, power[is_peak][peak_order[-1:-num_peaks-1:-1]]

In [ ]:
peaks, peak_power = get_peaks(1/freq, power)
peaks, peak_power

In [ ]:
selected_peak = peaks[4]
selected_peak

In [ ]:
ASASsin_g_ts = timeseries.TimeSeries(
    time=atime.Time(ASASsn_g['HJD'], format='jd', scale='utc'),
    data={'mag':np.array(ASASsn_g['mag']), 'mag_err':np.array(ASASsn_g['mag_err'])})
ASASsin_g_ts_fold = ASASsin_g_ts.fold(period=selected_peak)

ASASsin_g_ts_fold_binned = timeseries.aggregate_downsample(
    ASASsin_g_ts_fold, time_bin_size=12*u.hour, aggregate_func=np.nanmean)
ASASsin_g_ts_fold_binned_err = timeseries.aggregate_downsample(
    ASASsin_g_ts_fold, time_bin_size=12*u.hour, aggregate_func=lambda x: np.nanstd(x, ddof=1) / np.sqrt(len(x)))

fig, ax = plt.subplots(figsize=(5, 5), dpi=144)
ax.plot(ASASsin_g_ts_fold.time.jd, ASASsin_g_ts_fold['mag'], '.', markersize=1, color='gray', alpha=0.5)
ax.errorbar(
    ASASsin_g_ts_fold_binned.time_bin_start.jd,
    ASASsin_g_ts_fold_binned['mag'],
    ASASsin_g_ts_fold_binned_err['mag'],
    fmt='.',
)

ax.set_ylim(11.6, 11.2)
# ax.set_xlim(-8, -5)
ax.set_title('Folded ASASSN light curve of TOI-2000')
ax.set_ylabel('$g$ magnitude')

ax.set_xlabel('Days from arbitrary epoch')

ASASsin_g_ts

In [ ]:
fig = plt.figure(figsize=(8, 4.5), dpi=300)
gs = gridspec.GridSpec(2, 2)

ax = fig.add_subplot(gs[0, 0])
ax.plot(1/freq, power, zorder=100)
ax.set_xscale('log')
ax.set_ylim(-0.002, 0.034)
# ax.set_title('ASASSN Lomb-Scargle')
ax.hlines(ls_g.false_alarm_level([0.01, 0.001], method='bootstrap'), 0.5, 1500, linestyle=':', colors='gray')
ax.vlines([17.3, 90], -0.1, 0.4, linestyle='--', colors='purple')
ax.set_xlabel('')
ax.set_ylabel('Normalized power')
ax.text(0.05, 0.75, '$g$-band\nperiodogram', transform=ax.transAxes)
ax.tick_params(direction="in", which='both', labelsize="small", labelbottom=False, right=True, top=True)

ax = fig.add_subplot(gs[1, 0], sharex=ax)
ax.plot(1/freq, wn_power, zorder=100)
# ax.set_title('ASASSN window function')
ax.vlines([17.3, 90], -0.1, 0.42, linestyle='--', colors='purple')
ax.set_ylim(-0.002, 0.034)
ax.set_xlim(1.5, 365.25)
ax.set_xlabel('Period (day)')
ax.set_ylabel('Normalized power')
ax.text(0.05, 0.75, '$g$-band\nwindow function', transform=ax.transAxes)
ax.tick_params(direction="in", which='both', labelsize="small", right=True, top=True)

ax = fig.add_subplot(gs[0, 1])
ax.plot(1/freq_V, power_V, zorder=100)
ax.set_xscale('log')
ax.set_ylim(-0.002, 0.034)
# ax.set_title('ASASSN Lomb-Scargle')
ax.hlines(ls_V.false_alarm_level([0.01, 0.001], method='bootstrap'), 0.5, 1500, linestyle=':', colors='gray')
ax.vlines([17.3, 90], -0.1, 0.4, linestyle='--', colors='purple')
ax.set_xlabel('')
ax.set_ylabel('Normalized power')
ax.text(0.05, 0.75, '$V$-band\nperiodogram', transform=ax.transAxes)
ax.tick_params(direction="in", which='both', labelsize="small", labelbottom=False, right=True, top=True)

ax = fig.add_subplot(gs[1, 1], sharex=ax)
ax.plot(1/freq_V, wn_power_V, zorder=100)
# ax.set_title('ASASSN window function')
# ax.hlines(ls_V.false_alarm_level([0.01, 0.05]), 0.5, 1500, linestyle='--', colors='gray')
ax.vlines([17.3, 90], -0.1, 0.42, linestyle='--', colors='purple')
ax.set_ylim(-0.002, 0.034)
ax.set_xlim(1.5, 365.25)
ax.set_xlabel('Period (day)')
ax.set_ylabel('Normalized power')
ax.text(0.05, 0.75, '$V$-band\nwindow function', transform=ax.transAxes)
ax.tick_params(direction="in", which='both', labelsize="small", right=True, top=True)

# ax = fig.add_subplot(gs[:, 1])
# ax.plot(ASASsin_g_ts_fold.time.jd, ASASsin_g_ts_fold['mag'], '.', markersize=1, color='gray', alpha=0.5, rasterized=True)
# ax.errorbar(
#     ASASsin_g_ts_fold_binned.time_bin_start.jd,
#     ASASsin_g_ts_fold_binned['mag'],
#     ASASsin_g_ts_fold_binned_err['mag'],
#     fmt='.',
# )

# ax.set_ylim(11.49, 11.31)
# # ax.set_xlim(-8, -5)
# ax.text(0.05, 0.9, 'Folded ASAS-SN $g$-band light curve', transform=ax.transAxes)
# ax.text(0.05, 0.1, f'$P = {selected_peak.value:.2f}$ d', transform=ax.transAxes)
# # ax.set_title('Folded ASASSN light curve of TOI-2000')
# ax.set_ylabel('$g$ magnitude')

# ax.set_xlabel('Days from arbitrary epoch')

fig.suptitle('ASAS-SN light curves of TOI-2000', fontsize='medium', y=0.95)
fig.tight_layout()

fig.savefig('../plots/asassn_ls.pdf', bbox_inches='tight', pad_inches=0.01)
fig.savefig('../plots/asassn_ls.png', bbox_inches='tight', pad_inches=0.01, dpi=300)
fig.savefig('../plots/asassn_ls.svg', bbox_inches='tight', pad_inches=0.01)

In [ ]:
ASASsn_V = ASASsn_table.query('Filter == "V"')

ls_V = timeseries.LombScargle(
    t=atime.Time(ASASsn_V['HJD'], format='jd', scale='utc'),
    y=np.array(ASASsn_V['mag']),
    dy=np.array(ASASsn_V['mag_err']),
)

max_period = (np.max(ASASsn_V['HJD']) - np.min(ASASsn_V['HJD'])) * u.day
freq, power = ls_V.autopower(minimum_frequency=1/max_period)

fig, ax = plt.subplots(dpi=144)
ax.plot(1/freq, power)
ax.set_xscale('log')

ax.hlines(ls_V.false_alarm_level([0.01, 0.05, 0.1]), 0.2, 1000, colors='gray')
ls_V.false_alarm_level([0.01, 0.05, 0.1])

In [ ]:
np.max(ASASsn_V['HJD']) - np.min(ASASsn_V['HJD']), np.max(ASASsn_g['HJD']) - np.min(ASASsn_g['HJD'])